In [1]:
import pandas as pd  # For data manipulation and handling
import re  # For regular expressions

from tensorflow.keras.models import load_model  # To load trained models
from transformers import AutoTokenizer  # To use a pre-trained tokenizer (e.g., BERT)

import numpy as np  # If needed for numerical operations

hello


In [ ]:
# Tokenization using Hugging Face Transformers (BERT Tokenizer)
MODEL_NAME = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

max_length = 128  #

# Tokenize the text
def tokenize_texts(texts, tokenizer, max_length=max_length):
    return tokenizer(
        list(texts),
        max_length=max_length,
        truncation=True,
        padding='max_length',
        return_tensors='tf'
    )


In [ ]:
test_data = pd.read_csv('/kaggle/input/dataset/test.csv')

# Convert text to lowercase
test_data['Discussion'] = test_data['Discussion'].str.lower()
# Remove URLs and special characters
url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
test_data['Discussion'] = test_data['Discussion'].replace({r'\\n': ' '}, regex=True)
test_data['Discussion'] = test_data['Discussion'].apply(lambda x: re.sub(url_pattern, '', x))
test_data['Discussion'] = test_data['Discussion'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))
test_data['Discussion'] = test_data['Discussion'].apply(lambda x: x.strip())  # Remove leading/trailing spaces
test_data['Discussion'] = test_data['Discussion'].apply(lambda x: re.sub(r'\s+', ' ', x))  # Replace multiple spaces with a single space

In [ ]:
y_output_test = [test_data.copy()] * 6
# Load the trained model
for i in range(5):
  model = load_model(f'my_model{i}.h5')
  # Tokenize training and testing data
  Xt = tokenize_texts(test_data['Discussion'], tokenizer, max_length=max_length)
  # Extract token IDs for embedding input
  Xti = Xt['input_ids']
  y_output_test[i] = model.predict(Xti, verbose=1)
ttttt = [0] * len( y_output_test[0])
print(f'len(ttttt)= {len(ttttt)}')

for i in range(len(y_output_test[0])):
  mx, mxi = -1, 0
  for c in range(5):
    if mx < y_output_test[c][i]:
      mx = y_output_test[c][i]
      mxi = c
  ttttt[i] = mxi

In [ ]:
test_data['Category'] = ttttt

test_data['Category'].head(20)

# Save results to a new CSV file
output_file = 'predicted_categories.csv'
test_data[['SampleID', 'Category']].to_csv(output_file, index=False)

print(f"Predictions saved to {output_file}")